In [ ]:
from gatsbi.task_utils.benchmarks import make_generator
from gatsbi.optimize import BaseSR as Opt
from gatsbi.networks import WrapGenMultipleSimulations
import sbibm

In [ ]:
# Make task callables
task_name = "two_moons"
task = sbibm.get_task(task_name)
prior = task.get_prior()
simulator = task.get_simulator()


In [ ]:
# Make generator network
gen = make_generator(seq_impwts="impwts",
                     add_noise_kwargs={"lat_dim": 2, "output_dim": 128, "heteroscedastic": True},
                     add_noise_layer=5,
                     gen_nonlin="leaky_relu",
                     gen_nonlin_kwargs={"negative_slope": 0.1},
                     gen_units=[task.dim_data, 128, 128, 128, 128, task.dim_parameters])


In [ ]:
# wrap the generator so that it can generate multiple outputs for each input:
num_simulations_generator = 10
gen_wrapped = WrapGenMultipleSimulations(gen, n_simulations=num_simulations_generator)

In [ ]:
# Set training hyperparameters
training_opts = {
    "gen_iter": 1,
    "max_norm_gen": .1,
    "num_simulations": 100,
    "sample_seed": None,
    "hold_out": 10,
    "batch_size": 10,
    "log_dataloader": False,
    "stop_thresh": 0.001,
        }
gen_optim_args = [0.0001, [0.9, 0.99]]
scoring_rule = "energy_score"

In [ ]:
# Make optimizer
opt = Opt(
        generator=gen_wrapped,
        prior=prior,
        simulator=simulator,
        optim_args=[gen_optim_args],
        scoring_rule=scoring_rule,
        training_opts=training_opts
            )

In [6]:
# Make optimizer
opt = Opt(
        generator=gen_wrapped,
        prior=prior,
        simulator=simulator,
        optim_args=[gen_optim_args],
        scoring_rule=scoring_rule,
        training_opts=training_opts
            )

In [7]:
# Train
opt.train(10)

100%|#############################################################################################################################################################################| 10/10 [00:00<00:00, 162.88it/s]
